Hi, in this notebook you will learn the basic of the eager mode in Tensorflow 2. TensorFlow's eager execution is an imperative programming environment that evaluates operations immediately, without building graphs: operations return concrete values instead of constructing a computational graph to run later. This makes it easy to get started with TensorFlow and debug models, and it reduces boilerplate as well.

Why Tensorflor transform from graph execution to eager execution? Maybe it's the same reason why people like to code with Python and Jupyter Notebook: eager execution is much more intuitive and natural for programmers. A compasiron is made in this notebook by checking how these two modes perform the same operation.

In Tensorflow 2, eager mode is the default mode. If you still want to use graph execution, the session/graph/.. are under tf.compat.v1 module now. You will see a example of how to do things in Tensorflow 1.x style in this notebook.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
print(tf.__version__)

2.3.0


### The basic datatype in Tensorflow : Tensors

A Tensor is a multi-dimensional array. Similar to NumPy ndarray objects, Tensor objects have a data type and a shape. Additionally, Tensors can reside in accelerator (like GPU) memory.

In [2]:
# Create a tensor
one_dimentional_tensor = tf.constant([1,2,3,4])
print(one_dimentional_tensor)
a = tf.constant([[1,9],[3,6]])
print(a)
b = tf.constant([[3,3],[4,4]])
print(b)

tf.Tensor([1 2 3 4], shape=(4,), dtype=int32)
tf.Tensor(
[[1 9]
 [3 6]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[3 3]
 [4 4]], shape=(2, 2), dtype=int32)


In [3]:
# Get its numpy
a.numpy()

array([[1, 9],
       [3, 6]])

In [4]:
# Get its list
a.numpy().tolist()

[[1, 9], [3, 6]]

### Operations and Control Flow

 TensorFlow offers a rich library of operations (tf.add, tf.matmul, tf.linalg.inv etc.) that consume and produce Tensors. 

In [5]:
tf.add(a, b)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 4, 12],
       [ 7, 10]])>

In [6]:
a + b

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 4, 12],
       [ 7, 10]])>

These operations automatically convert native Python types. For example:

In [7]:
print(tf.subtract(a , b))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

tf.Tensor(
[[-2  6]
 [-1  2]], shape=(2, 2), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)


Pythonic control flow is supported. A major benefit of eager execution is that all the functionality of the host language is available while your model is executing

In [8]:
cond_v = tf.constant(10)
if cond_v > 8:
    c = tf.add([1, 2], [1, 2])
else:
    c = tf.subtract([1, 2], [1, 2])
print(c)

tf.Tensor([2 4], shape=(2,), dtype=int32)


### Graph and tf.funciton

How to perform the operation above in Tensorflow 1.x? You have to define a graph, put all needed operations and variables into the graph, then create a session to require specified output from the graph, optionally with given values on some varibles in the graph. 

In [9]:
g = tf.compat.v1.Graph()
with g.as_default():
    cond_v = tf.constant(9)
    x = tf.constant([1,2])
    y = tf.constant([1,2])
    z = tf.add(x, y)
    z2 = tf.subtract(x, y)
    result = tf.cond(cond_v>8, lambda: z, lambda: z2)
with tf.compat.v1.Session(graph=g) as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    print(sess.run(result))
    print(sess.run(result, feed_dict={cond_v : 7}))

[2 4]
[0 0]


The built graph is as follows:

<img src="resources/graph.PNG" alt="drawing" width="500" align="left"/>

Graph execution may have some advantages. With your operations represented as a platform-independent graph, computation like differentiation will be optimized and automated. Moreover, you can deploy the platform-independent graph on python-free server and various kinds of devices, such as smartphones.

In tensorflow 2, the power of graph is reversed by tf.function, which allows you to transform a subset of Python syntax into portable, high-performance TensorFlow graphs.

In [10]:
@tf.function
def add_or_subtract(cond_v):
    if cond_v > 8:
        return tf.add([1, 2], [1, 2])
    else:
        return tf.subtract([1, 2], [1, 2])

In [11]:
print(add_or_subtract(tf.constant(9)))
print(add_or_subtract(tf.constant(7)))

tf.Tensor([2 4], shape=(2,), dtype=int32)
tf.Tensor([0 0], shape=(2,), dtype=int32)
